<a href="https://colab.research.google.com/github/RylieWeaver/Hyperparameter-Optimization/blob/main/Population_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Population Propagation (Strategy) Comparison in DEAP (Distributed Evolutionary Algorithms in Python)

Compare the performance of the `simple` and `mu_plus_lambda` strategies in DEAP (w/ controlled number of evalutions). These strategies control how offspring and parents are carried into the next generation in the genetic algorithm. In a nutshell, `mu_plus_lambda` carries over the parents while `simple` does not.

### Setup

In [1]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


In [2]:
import random
from deap import base, creator, tools, algorithms
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [3]:
n=16
cxpb=0.2
cx_indpb=0.5
mutpb=0.8
mut_indpb=1
tournsize=4

### Strategies

In [4]:
def mu_plus_lambda():

    pop = toolbox.population(n)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Parameters for the mu+lambda algorithm
    mu = len(pop)
    lambda_ = round(0.5*len(pop)) # Offspring is half the size of population

    pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu, lambda_, cxpb, mutpb,  # mutpb, cxpb is the mutation and crossover probability
                                         ngen=5, stats=stats, halloffame=hof, verbose=False)

    return pop, log, hof

# pop, log, hof = mu_plus_lambda()

# Print best individual
# best_individual = (hof[0], hof[0].fitness.values[0])
# print("Best individual is: %s\nWith fitness: %s" % best_individual)

In [5]:
def simple():
    # Create initial population
    pop = toolbox.population(int(n/2))
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb, mutpb,  # mutpb, cxpb is the mutation and crossover probability
                                   ngen=6, stats=stats, halloffame=hof, verbose=False)

    return pop, log, hof

# pop, log, hof = simple()

# Print best individual
# best_individual = (hof[0], hof[0].fitness.values[0])
# print("Best individual is: %s\nWith fitness: %s" % best_individual)

### OneD

In [6]:
params = []
fitnesses = []

def fitness(individual):
    x = individual[0]
    y = 0.02 * x + 0.5 * np.sin(1 * x + 0.1) + 0.75 * np.cos(0.25 * x - 0.3)

    # Log the parameters and fitness for plotting
    params.append(x)
    fitnesses.append(y)

    return y,

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 20)  # Create random numbers between 0 and 20
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the evaluation function
toolbox.register("evaluate", fitness)

# Make decorator to keep mutation within bounds
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] > max:
                        child[i] = max
                    elif child[i] < min:
                        child[i] = min
            return offspring
        return wrapper
    return decorator

toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=mut_indpb)  # Gaussian mutation
toolbox.decorate("mutate", checkBounds(0, 20))  # Keep mutation within bounds
toolbox.register("mate", tools.cxUniform, indpb=cx_indpb)  # indpb is the probability of each attribute to be exchanged
toolbox.register("select", tools.selTournament, tournsize=tournsize) # Selection operator

### 2D Sphere

In [9]:
params1 = []
params2 = []
fitnesses = []

def fitness(individual):
    x1 = individual[0]
    x2 = individual[1]
    y = x1**2 + x2**2

    # Log the parameters and fitness for plotting
    params1.append(x1)
    params2.append(x2)
    fitnesses.append(y)

    return (y),

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, -5, 5)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxUniform, indpb=cx_indpb)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=mut_indpb)
toolbox.register("select", tools.selTournament, tournsize=tournsize)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Rastigin

In [12]:
params1 = []
params2 = []
fitnesses = []

def fitness(individual):
    x1 = individual[0]
    x2 = individual[1]
    y = 20 + ((x1/np.pi)**2 - 10 * np.cos(2 * (x1/np.pi))) + ((x2/np.pi)**2 - 10 * np.cos(2 * (x2/np.pi)))

    # Log the parameters and fitness for plotting
    params1.append(x1)
    params2.append(x2)
    fitnesses.append(y)

    return (y),

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxUniform, indpb=cx_indpb)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=mut_indpb)
toolbox.register("select", tools.selTournament, tournsize=tournsize)

### Beale

In [15]:
params1 = []
params2 = []
fitnesses = []

def fitness(individual):
    x1 = individual[0]
    x2 = individual[1]
    term1 = (1.5 - x1 + x1 * x2) ** 2
    term2 = (2.25 - x1 + x1 * x2 ** 2) ** 2
    term3 = (2.625 - x1 + x1 * x2 ** 3) ** 2
    y = term1 + term2 + term3

    # Log the parameters and fitness for plotting
    params1.append(x1)
    params2.append(x2)
    fitnesses.append(y)

    return (y),

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxUniform, indpb=cx_indpb)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=mut_indpb)
toolbox.register("select", tools.selTournament, tournsize=tournsize)

### Himmelblau's

In [18]:
params1 = []
params2 = []
params3 = []
fitnesses = []

def fitness(individual):
    x1 = individual[0]
    x2 = individual[1]
    x3 = individual[2]
    term1 = (x1 ** 2 + x2 - 17) ** 2
    term2 = (x1 + x2 ** 2 - 4) ** 2
    term3 = (x1 ** 2 + x3 - 11) ** 2
    term4 = (x1 + x3 ** 2 + 13) ** 2
    y = term1 + term2 + term3

    # Log the parameters and fitness for plotting
    params1.append(x1)
    params2.append(x2)
    params3.append(x3)
    fitnesses.append(y)

    return (y),

# Define the individual and the population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxUniform, indpb=cx_indpb)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=3, indpb=mut_indpb)
toolbox.register("select", tools.selTournament, tournsize=tournsize)

### Compare Average

OneD:

In [7]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = mu_plus_lambda()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: -0.8430130231743078
Median Min: -0.8547098181438396
Min STD: 0.03377832289275117
Average Max: -0.7850955096631104
Median Max: -0.8337742113412155
Max STD: 0.12469755800715975


In [8]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = simple()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: -0.8426792711633881
Median Min: -0.855078616581745
Min STD: 0.035279305428873434
Average Max: 0.47060147291465954
Median Max: 0.5538483958511367
Max STD: 0.32672940402006445


2D Sphere:

In [10]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = mu_plus_lambda()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 0.4884895725859049
Median Min: 0.3218446636109639
Min STD: 0.5168494019006074
Average Max: 1.8202852615250085
Median Max: 1.085942388550384
Max STD: 2.8041172374192813


In [11]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = simple()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 0.5692157688963475
Median Min: 0.37229657542566064
Min STD: 0.6208141535886474
Average Max: 57.73519142057723
Median Max: 52.46739949478842
Max STD: 29.384859909845513


Rastigin:

In [13]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = mu_plus_lambda()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 3.0619208040487815
Median Min: 1.6349575217752363
Min STD: 3.39516865897393
Average Max: 7.205787505620005
Median Max: 6.597815933208679
Max STD: 5.086729748877683


In [14]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = simple()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 3.857030471288724
Median Min: 1.9451428197398677
Min STD: 4.042509064870627
Average Max: 38.999892843137516
Median Max: 38.51663491181185
Max STD: 9.515172756701341


Beale:

In [16]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = mu_plus_lambda()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 3.4807100273126066
Median Min: 1.5675463910144405
Min STD: 15.800073095569168
Average Max: 426.21606503783204
Median Max: 8.454185736491208
Max STD: 14221.619691199054


In [17]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = simple()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 3.773292483763123
Median Min: 1.4919236261128623
Min STD: 27.631635669386235
Average Max: 3401796.501326777
Median Max: 308622.748592195
Max STD: 16734796.546130804


HimmelBlau:

In [19]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = mu_plus_lambda()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 38.41732252526253
Median Min: 31.054906537632647
Min STD: 30.669673449531466
Average Max: 109.1181770508948
Median Max: 78.12132977674997
Max STD: 179.32201112952848


In [20]:
min_scores = []
max_scores = []

for i in range(5000):
  pop, log, hof = simple()
  min_scores.append(hof[0].fitness.values[0])
  max_scores.append(log[-1]['max'])

print(f"Average Min: {np.mean(min_scores)}")
print(f"Median Min: {np.median(min_scores)}")
print(f"Min STD: {np.std(min_scores)}")
print(f"Average Max: {np.mean(max_scores)}")
print(f"Median Max: {np.median(max_scores)}")
print(f"Max STD: {np.std(max_scores)}")

Average Min: 40.945985760253265
Median Min: 32.55403223372173
Min STD: 33.31063817045386
Average Max: 7794.94501535162
Median Max: 4738.526902553187
Max STD: 9479.879750571885


In these semi_simple cases, the `mu_plus_lambda` strategy outperforms the `simple` strategy.